# 3. Calcolo di Metriche Avanzate e Normalizzazione

**Obiettivo:** Arricchire il dataset con metriche che offrono una visione più profonda delle performance dei giocatori, superando i limiti delle statistiche tradizionali.

**Fasi:**
1.  **Caricamento Dati:** Lettura del DataFrame pulito.
2.  **Normalizzazione 'Per 36 Minuti':** Standardizzazione delle statistiche per permettere un confronto equo tra giocatori con minutaggi diversi (es. titolari vs. riserve).
3.  **Calcolo del True Shooting Percentage (TS%):** Calcolo di una metrica avanzata per misurare l'efficienza reale al tiro, tenendo conto di tiri da due, da tre e tiri liberi.
4.  **Salvataggio:** Memorizzazione del DataFrame arricchito.

In [ ]:
import os
import sys

# Aggiunge la root del progetto al sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Importazione delle utility e delle funzioni specifiche
from src.utils.helpers import get_spark_session, save_dataframe
from src.data_processing.normalization import per_36_minutes_stats
from src.feature_engineering.advanced_metrics import calculate_true_shooting_percentage
from src.config.spark_config import SPARK_CONFIG

# Inizializzazione della sessione Spark
spark = get_spark_session(
    app_name="NBA_Advanced_Metrics",
    driver_memory=SPARK_CONFIG["driver_memory"]
)

### Fase 1: Caricamento dei Dati Puliti

In [ ]:
# Caricamento del DataFrame pulito
cleaned_df_path = "../data/processed/players_cleaned.parquet"
df = spark.read.parquet(cleaned_df_path)

### Fase 2: Normalizzazione delle Statistiche (Per 36 Minuti)

Questa normalizzazione proietta le statistiche di ogni giocatore su un tempo di gioco standard, rendendo possibile confrontare l'impatto di un giocatore di rotazione con quello di un titolare.

In [ ]:
# Elenco delle statistiche da normalizzare
stats_to_normalize = ['pts', 'trb', 'ast', 'stl', 'blk', 'tov', 'fga', 'fta', 'fg', 'ft']

# Applica la funzione di normalizzazione
df_normalized = per_36_minutes_stats(df, stats_to_normalize, minutes_played_col="mp")

# Visualizzazione di un campione delle statistiche normalizzate
print("Esempio di statistiche normalizzate per 36 minuti:")
df_normalized.select('player', 'season', 'pts_per_36_min', 'trb_per_36_min', 'ast_per_36_min').show(10)

### Fase 3: Calcolo Metriche Avanzate (True Shooting %)

Il TS% è una misura di efficienza superiore alla semplice percentuale dal campo (FG%) perché assegna un peso corretto ai tiri da tre punti e ai tiri liberi.

In [ ]:
# Calcola il True Shooting Percentage (TS%)
df_advanced = calculate_true_shooting_percentage(df_normalized, points_col="pts", fga_col="fga", fta_col="fta")

# Visualizza i giocatori con la più alta efficienza al tiro calcolata
print("Top 10 Giocatori per True Shooting Percentage (calcolato):")
df_advanced.select('player', 'season', 'pts_per_36_min', 'ts_pct_calc') \
             .orderBy(df_advanced.ts_pct_calc.desc()) \
             .show(10)

### Fase 4: Salvataggio del DataFrame Arricchito

In [ ]:
# Definisce il percorso di output e salva il DataFrame
output_path = "../data/processed/players_advanced_metrics.parquet"
save_dataframe(df_advanced, output_path)

print(f"Dati con metriche avanzate salvati con successo in '{output_path}'.")

In [ ]:
# Termina la sessione Spark
spark.stop()